Import libraries required

In [10]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

Read data in

In [2]:
data = pd.read_csv('../data/total_fixed.csv', parse_dates=['Changed', 'Opened'])
cols = data.columns

n_rows = len(data)
n_cols = len(cols)
bugIDs = data['Bug ID']
print data.head()
print bugIDs.head()

   Bug ID   Product Component                  Assignee  Status Resolution  \
0    1868  Platform        UI  Kevin_Haaland@ca.ibm.com  CLOSED      FIXED   
1    2072  Platform        UI  Kevin_Haaland@ca.ibm.com  CLOSED      FIXED   
2    2583  Platform        UI  Kevin_Haaland@ca.ibm.com  CLOSED      FIXED   
3    2630  Platform        UI  Kevin_Haaland@ca.ibm.com  CLOSED      FIXED   
4    2637  Platform        UI  Kevin_Haaland@ca.ibm.com  CLOSED      FIXED   

                                 Summary             Changed Priority  \
0             Task Smoke Tests (1GHE30L) 2001-10-10 22:21:00       P3   
1      Preferences Smoke Tests (1GIIP94) 2001-10-10 22:25:00       P3   
2        Navigator Smoke Tests (1GHDUB4) 2001-10-10 22:39:00       P3   
3          Outline Smoke Tests (1GHFYSV) 2001-10-10 22:40:00       P3   
4  Properties View Smoke Tests (1GHFYQ0) 2001-10-10 22:40:00       P3   

  Severity Keywords                              Summary.1              Opened  
0   normal 

In [3]:
print data['Changed'].head()
print set(data['Severity'])

severity_levels = {'enhancement': 0, 'trivial' : 1, 'minor' : 2, 'normal' : 3, 'major' : 4, 'critical' : 5, 'blocker' : 6}

0   2001-10-10 22:21:00
1   2001-10-10 22:25:00
2   2001-10-10 22:39:00
3   2001-10-10 22:40:00
4   2001-10-10 22:40:00
Name: Changed, dtype: datetime64[ns]
set(['major', 'normal', 'blocker', 'critical', 'enhancement', 'trivial', 'minor'])


In [66]:
np_opened = np.array(data['Opened'])
np_opened = np_opened.reshape(np_opened.shape[0], 1)
np_opened.shape

(103805, 1)

In [57]:
# Generating Temporal Features
def tmp1(x, n):
    return sum(np_opened < x) - sum (np_opened < (x - np.timedelta64(n, 'D')))

def tmp2(x, n):
    subdata = data[data['Severity'] == x['Severity']]
    return sum(subdata['Opened'] < x['Opened']) - sum (subdata['Opened'] < (x['Opened'] - timedelta(days=n)))

def subdatagen(x, severity):
    if severity_levels[x['Severity']] >= severity_levels[severity]:
        return True
    else:
        return False

def tmp3(i, n):
    condn = data.apply(lambda x: subdatagen(x, i['Severity']), axis=1)
    subdata = data[condn]
    return sum(subdata['Opened'] < x['Opened']) - sum (subdata['Opened'] < (x['Opened'] - timedelta(days=n)))

In [62]:
# data['Opened'].map(lambda x: tmp1(x, 7))
np_opened[0] - np.timedelta64(7, 'D')

numpy.datetime64('2001-10-03T22:21:00.000000000')

In [63]:
tmp1(np_opened[0], 7)

655

In [38]:
sum(np_opened < np_opened[0])[0]

655

In [67]:
np.apply_along_axis(lambda x: tmp1(x, 7), 1, np_opened)

KeyboardInterrupt: 